# Observed Taylor law: variance vs mean 
**in the protein coding gene length distributions for the different species**

## Import python modules

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from plotnine import *

import sys
sys.path.append('../../gene_length/gl_lib/')
import gl_constants as c
import gl_plot as gl_p

## The protein coding gene length distributions for the different species

### Retrieving the statistical descriptions of the gene length's distributions

#### Statistical description 

In [2]:
# statistics on length distributions 
stat_file = "/home/emuro/git/github/gene_length/main_tables/stat_protCodGenes.tsv"
print("The statistical descriptions of the protein coding gene distributions " + \
    "for the different species is in:\n", stat_file, "\n")

# retrieve data and diminish the number of columns
stat_df = pd.read_csv(stat_file, low_memory=False, sep="\t")
stat_df = stat_df[["division_both_dbs", "species", "taxonomy_id","count", "mean", "var", "log10_mean", "log10_var"]]

# visualize data
if 1:
    pd.set_option('display.max_columns', len(stat_df.columns))
    display(stat_df.sample(2))
    print(stat_df.shape)
    print(stat_df["division_both_dbs"].value_counts())

The statistical descriptions of the protein coding gene distributions for the different species is in:
 /home/emuro/git/github/gene_length/main_tables/stat_protCodGenes.tsv 



,division_both_dbs,species,taxonomy_id,count,mean,var,log10_mean,log10_var
11800,bacteria,pedobacter_africanus_gca_900176535,151894,4708,1129.215803,717714.302686,2.949478,0.093759
29401,bacteria,rhizobium_sp_pp_cc_2g_626_gca_004341455,2135646,5518,936.955237,408292.710883,2.879473,0.088327


(33627, 8)
division_both_dbs
bacteria         30714
archaea           1229
fungi             1014
protists           237
vertebrates        222
invertebrates      115
plants              96
Name: count, dtype: int64


## Get the genome size  

In [3]:
# statistics on length distributions 
stat2_file = "/home/emuro/git/github/bioinf-genome_length/ensembl__taxid_species_goldenPath_division.tsv.bz2"
print("The genome sizes in bp, that I downloaded, " + \
    "for the different species is in:\n", stat2_file, "\n")

# retrieve data and diminish the number of columns
stat2_df = pd.read_csv(stat2_file, compression="bz2", low_memory=False, sep="\t")
#stat2_df = stat_df[["division_both_dbs", "species", "taxonomy_id","count", "mean", "var", "log10_mean", "log10_var"]]
stat2_df = stat2_df.rename(columns={"taxid": "taxonomy_id", "golden_path": "genome"})
# visualize data
if 1:
    pd.set_option('display.max_columns', len(stat2_df.columns))
    display(stat2_df.sample(2))
    print(stat2_df.shape)
    print(stat2_df["division"].value_counts())
print(stat_df["division_both_dbs"].value_counts())

The genome sizes in bp, that I downloaded, for the different species is in:
 /home/emuro/git/github/bioinf-genome_length/ensembl__taxid_species_goldenPath_division.tsv.bz2 



,taxonomy_id,species,genome,division
24047,2026779,planctomycetaceae_bacterium_gca_003503535,6562623,EnsemblBacteria
19291,345309,luteibacter_yeojuensis_gca_000964085,4548902,EnsemblBacteria


(33022, 4)
division
EnsemblBacteria       31332
EnsemblFungi           1014
EnsemblProtists         237
EnsemblVertebrates      227
EnsemblMetazoa          115
EnsemblPlants            96
EnsemblViruses            1
Name: count, dtype: int64
division_both_dbs
bacteria         30714
archaea           1229
fungi             1014
protists           237
vertebrates        222
invertebrates      115
plants              96
Name: count, dtype: int64


## Merge genome size column

In [4]:
# merge data frames
stat_df = (
    pd.merge(
        stat_df, stat2_df, 
        how='left', 
        left_on=['taxonomy_id', 'species'], right_on=['taxonomy_id', 'species']
        )
        .fillna({'genome':'NaN'})
)
# Counting NaN values in the 'golden_path' column
#print(stat2_df['genome'].isna().sum())


# visualize data
if 1:
    print("stat_df")
    pd.set_option('display.max_columns', len(stat_df.columns))
    display(stat_df.sample(2))
    print(stat_df.shape)

0
stat_df


,division_both_dbs,species,taxonomy_id,count,mean,var,log10_mean,log10_var,genome,division
10063,bacteria,meiothermus_silvanus_gca_004152925,52022,3206,928.545540,3.652970e+05,2.887623,0.072469,3342742,EnsemblBacteria
32749,fungi,cadophora_sp_dse1049_gca_003073865,1485229,22762,1581.999341,1.235757e+06,3.110778,0.081698,70459101,EnsemblFungi


(33630, 10)


## mean vs. genome size (represented in $log_{10}$ scale)
### Protein coding gene's length

In [5]:
col_x = "mean"  
col_y = "genome"  
legends_by = "division_both_dbs"

title = "Protein coding genes" # "Uniprot, reference Proteomes (protein length)"
x_lab = "Mean gene length"   
y_lab = "Genome (bp)"
bool_show_regression = True

df2plot = stat_df
if 0:
    print(df2plot.shape)

# Sort division_both_dbs in order to plot the color of the clades in order
if 0:
    print(df2plot.division_both_dbs.unique())
df2plot.division_both_dbs = pd.Categorical(df2plot.division_both_dbs, 
                                categories=c.ORG_GROUPS,           
                                ordered=True)
df2plot.sort_values('division_both_dbs', inplace=True)

col_x = "mean"

p = (   
        ggplot(df2plot, aes(col_x, col_y, color=legends_by)) + 
        geom_point(size=0.15) + #, alpha=0.4 + # color, fill )
        geom_smooth(method="lm", color="green", size=0.25, span=.8) +
        labs(title=title, x=x_lab, y=y_lab) 
        + scale_color_manual(values=c.COLOR_ORG_GROUPS) 
        + labs(color='Group of organisms') # legend title
        + scale_x_log10(breaks=[10 ** power for power in range(6)],
          limits=[min(df2plot[col_x].to_list())/2, 2*max(df2plot[col_x].to_list())]) 
        + scale_y_log10(breaks = [10**power for power in range(13)], 
          limits = [min(df2plot[col_y].to_list())/2,2*max(df2plot[col_y].to_list())])#, labels=scientific_format(digits=2)
    ) + theme(legend_position=(0.95,0.1), legend_key_size=2, \
        legend_background=element_rect(fill='grey', alpha=0.01)) + \
            annotate('text', x=0.01*max(df2plot[col_x].to_list()), \
                y=0.65*max(df2plot[col_y].to_list()), label=txt,size=9,color="black")
    p.show()


#gl_p.plot_taylor_genes(df2plot, col_x, col_y, x_lab, y_lab, title, legends_by, bool_show_regression)
if 1:
    pd.set_option('display.max_columns', len(df2plot.columns))
    display(df2plot.sample(2))
    print(df2plot.shape)
    #print(df2plot["division"].value_counts())
    print(df2plot["division_both_dbs"].value_counts())

IndentationError: unexpected indent (1658440247.py, line 39)

#### Note:  
$\sigma^{2} = a . \mu^{\beta}$   
Because the regression line is:  
$\log(\sigma^{2}) = \log(a) + \beta log(\mu) $